### 题目
https://reformship.github.io/pages/1capacity/1model/%E6%A8%A1%E5%9E%8B%20%E4%BC%98%E5%8C%96%E6%A8%A1%E5%9E%8B30%20%E9%A3%9F%E7%94%A8%E6%B2%B9%E7%B2%BE%E7%82%BC%E9%97%AE%E9%A2%98.html  
个人觉得上面链接所给的答案有点问题，没有考虑到期货交易

In [ ]:
import pulp as lp

EveryMonth_prices = {
    'Jan': [110, 120, 130, 110, 115],
    'Feb': [130, 130, 110, 90, 115],
    'Mar': [110, 140, 130, 100, 95],
    'Apr': [120, 110, 120, 120, 125],
    'May': [100, 120, 150, 110, 105],
    'Jun': [90, 100, 140, 80, 135]
}
mon_index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']

# 硬度
hardness = [8.8, 6.1, 2.0, 4.2, 5.0]
max_hardness = 6
min_hardness = 3

# 初始化

# 储存每个月原料油的剩余
mon_storage = {}

# 每一个月的精炼油量
mon_refined_oil = {}

# 每一个月的成本
mon_cost = {}

# 期货的油量购买/对应当月到手的/其实是上一个月购买的
mon_futures = {}

# 当月油量购买
mon_raw_oil_buy ={}

# 定义每一个月的变量/5*3*6

for month in mon_index:

    if month == 'Jan':

        next_mon = mon_index[mon_index.index(month) + 1]

        initial_store = 500

        # 定义变量
        refined_oil = [lp.LpVariable(f'y_{month}_{i + 1}', lowBound=0,cat='Continuous') for i in range(5)]
        raw_oil_now = [lp.LpVariable(f'm_{month}_{i + 1}', lowBound=0, cat='Continuous') for i in range(5)]
        raw_oil_next = [lp.LpVariable(f'n_{next_mon}_{i + 1}', lowBound=0, cat='Continuous') for i in range(5)]
        
        sum_raw_oil_cost = lp.lpSum(EveryMonth_prices[month][i] * (raw_oil_now[i] + raw_oil_next[i]) for i in range(5))


        # 每月储存原油量
        storage = []
        for i in range(5):
            oil = initial_store - refined_oil[i] + raw_oil_now[i]
            storage.append(oil)

        mon_storage[month] = storage

        # 成本
        cost = []
        oil_cost = lp.lpSum(mon_storage[month]) * 5 + sum_raw_oil_cost
        cost.append(oil_cost)
        mon_cost[month] = cost


        # 期货总量
        mon_futures[next_mon] = raw_oil_next

        # 当月油量购买
        mon_raw_oil_buy[month] = raw_oil_now

        # 精油量
        mon_refined_oil[month] = refined_oil



    elif month != 'Jun' and month != 'Jan':

        next_mon = mon_index[mon_index.index(month) + 1]
        last_mon = mon_index[mon_index.index(month) - 1]

        # 定义变量
        refined_oil = [lp.LpVariable(f'y_{month}_{i + 1}', lowBound=0, cat='Continuous') for i in range(5)]
        raw_oil_now = [lp.LpVariable(f'm_{month}_{i + 1}', lowBound=0, cat='Continuous') for i in range(5)]
        raw_oil_next = [lp.LpVariable(f'n_{next_mon}_{i + 1}', lowBound=0, cat='Continuous') for i in range(5)]

        # 下个月的期货量
        mon_futures[next_mon] = raw_oil_next

        # 当月油量购买
        mon_raw_oil_buy[month] = raw_oil_now

        # 精油量
        mon_refined_oil[month] = refined_oil

        # 这个月购买的油量+上一个月期货到的油量
        sum_raw_oil_now = lp.lpSum(raw_oil_now) + lp.lpSum(mon_futures[month])
        sum_raw_oil_next = lp.lpSum(raw_oil_next)
        sum_raw_oil_cost = lp.lpSum(EveryMonth_prices[month][i] * (raw_oil_now[i] + raw_oil_next[i]) for i in range(5))

        # 储存原油量(油量剩余量)
        storage = []
        for i in range(5):
            # 原库存-精油量+当月购买的原油量+当月到手的期货量
            oil = mon_storage[last_mon][i] - refined_oil[i] + raw_oil_now[i] + mon_futures[month][i]
            storage.append(oil)
        mon_storage[month] = storage

        # 每个月成本=储存成本+购买成本(期货+现货)
        cost = []
        oil_cost = lp.lpSum(mon_storage[month]) * 5 + sum_raw_oil_cost
        cost.append(oil_cost)
        mon_cost[month] = cost





    elif month == 'Jun':

        last_mon = mon_index[mon_index.index(month) - 1]

        refined_oil = [lp.LpVariable(f'y_{month}_{i + 1}', lowBound=0, cat='Continuous') for i in range(5)]
        raw_oil_now = [lp.LpVariable(f'm_{month}_{i + 1}', lowBound=0, cat='Continuous') for i in range(5)]
        sum_raw_oil_cost = lp.lpSum(EveryMonth_prices[month][i] * raw_oil_now[i] for i in range(5))

        # 储存的油量
        storage = []
        for i in range(5):
            oil = mon_storage[last_mon][i] - refined_oil[i] + raw_oil_now[i] + mon_futures[month][i]
            storage.append(oil)
        mon_storage[month] = storage

        # 成本
        cost = []
        oil_cost = sum_raw_oil_cost + 5 * lp.lpSum(mon_storage[month])
        cost.append(oil_cost)
        mon_cost[month] = cost

        # 精油量
        mon_refined_oil[month] = refined_oil

        # 无下一个月的期货量

        # 当月油量购买
        mon_raw_oil_buy[month] = raw_oil_now

# 求和
every_mon_refined_oil = []
every_mon_cost = []
every_raw_oil_buy_now = []
every_mon_futures = []
all_mon_raw_oil_storage = []

for month in mon_index:
    oil = lp.lpSum(mon_refined_oil[month])
    cost = lp.lpSum(mon_cost[month])
    raw_oil_buy = lp.lpSum(mon_raw_oil_buy[month])

    if month == 'Jan':
        pass
    else:
        mon_future_oil = lp.lpSum(mon_futures[month])
        every_mon_futures.append(mon_future_oil)

    every_mon_refined_oil.append(oil)
    every_mon_cost.append(cost)
    all_mon_raw_oil_storage.append(mon_storage[month])  
    every_raw_oil_buy_now.append(raw_oil_buy)


total_refined_oil = lp.lpSum(every_mon_refined_oil)
total_cost = lp.lpSum(every_mon_cost)
total_raw_oil_buy_now = lp.lpSum(every_raw_oil_buy_now)
total_raw_oil_storage = lp.lpSum(all_mon_raw_oil_storage)
total_futures_oil = lp.lpSum(every_mon_futures)


# 表达式
print('total_refined_oil:\n',total_refined_oil)
print('-'*50)
print('total_cost:\n',total_cost)
print('-'*50)
print('total_raw_oil_buy_now:\n',total_raw_oil_buy_now)
print('-'*50)
print('total_raw_oil_storage:\n',total_raw_oil_storage)
print('-'*50)
print('total_futures_oil:\n',total_futures_oil)


# 目标函数
max_profit = lp.LpProblem('max_profit', sense=lp.LpMaximize)
max_profit += 150 * total_refined_oil - total_cost

VEG_cap = 200
OIL_cap = 250
max_cap = 1000
initial_cap = 500


#约束条件

for month in mon_index:

    # 储存量约束(每种原油储量<=1000,except Jun)
    if month == 'Jun':
        for i in range(5):
            max_profit += mon_storage[month][i] == initial_cap
    else:
        for i in range(5):
            max_profit += mon_storage[month][i] <= max_cap
            max_profit += mon_storage[month][i] >= 0

    # 精炼油约束
    max_profit += mon_refined_oil[month][0] + mon_refined_oil[month][1] <= VEG_cap
    max_profit += mon_refined_oil[month][2] + mon_refined_oil[month][3] + mon_refined_oil[month][4] <= OIL_cap

    # 硬度约束
    sum_refined_oil = lp.lpSum(mon_refined_oil[month])
    sum_refined_oil_hard_limit = []
    for i in range(5):
        refined_oil_hard_limit = hardness[i] * mon_refined_oil[month][i]
        sum_refined_oil_hard_limit.append(refined_oil_hard_limit)

    max_profit += lp.lpSum(sum_refined_oil_hard_limit) >= min_hardness * sum_refined_oil
    max_profit += lp.lpSum(sum_refined_oil_hard_limit) <= max_hardness * sum_refined_oil



print('max_profit:\n',max_profit)
print('-'*50)
print('mon_storage\n',mon_storage)
print('-'*50)
print('mon_futures:\n',mon_futures)
print('-'*50)
print('mon_refined_oil:\n',mon_refined_oil)
print('-'*50)
print('mon_cost:\n',mon_cost)

# 求解

max_profit.solve()
print("Status:", lp.LpStatus[max_profit.status])
print("max_profit", lp.value(max_profit.objective))

# 输出每个变量的最优值
for v in max_profit.variables():
    print(v.name, "=", v.varValue)


for month in mon_storage:
    print(f"{month} storage:")
    for i, oil in enumerate(mon_storage[month]):
        print(f"  Oil {i+1}: {lp.value(oil)}")
    
    print(f"{month} refined oil:")
    for i, oil in enumerate(mon_refined_oil[month]):
        print(f"  Oil {i+1}: {lp.value(oil)}")
    